# Modelo Predivo dos Rappitors

### Objetivo

Antecipar falhas críticas no sistema, como erros ou interrupções, com base no histórico de logs. 

A ideia é prever a probabilidade de ocorrer um evento com LogLevel = ERROR ou FATAL, considerando o comportamento anterior do sistema. Como uma espécie de "histórico de sintomas" antes de uma falha.

In [1]:
# importando o dataset do KaggleHub

import kagglehub

path = kagglehub.dataset_download("shubhampatil1999/synthetic-log-data-of-distributed-system")
# https://www.kaggle.com/datasets/shubhampatil1999/synthetic-log-data-of-distributed-system

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Inteli\.cache\kagglehub\datasets\shubhampatil1999\synthetic-log-data-of-distributed-system\versions\1


In [2]:
## Verificando os arquivos baixados
import os

caminho = r"C:\Users\Inteli\.cache\kagglehub\datasets\shubhampatil1999\synthetic-log-data-of-distributed-system\versions\1"
arquivos = os.listdir(caminho)

print(arquivos)

['logdata.csv']


In [3]:
# Importando o dataset
import pandas as pd

caminho_arquivo = r"C:\Users\Inteli\.cache\kagglehub\datasets\shubhampatil1999\synthetic-log-data-of-distributed-system\versions\1\logdata.csv"
df = pd.read_csv(caminho_arquivo)

## Exploração de dados

In [4]:
df.head(20)

,Unnamed: 0,Timestamp,LogLevel,Service,Message,RequestID,User,ClientIP,TimeTaken
0,0,2023-11-20T08:40:50.664842,WARNING,ServiceA,Performance Warnings,6743,User96,192.168.1.102,28ms
1,1,2023-11-20T08:40:50.672154,DEBUG,ServiceA,File I/O,8684,User17,192.168.1.219,55ms
2,2,2023-11-20T08:40:50.680263,WARNING,ServiceA,Performance Warnings,6265,User13,192.168.1.173,44ms
3,3,2023-11-20T08:40:50.688973,ERROR,ServiceA,Critical Errors,5821,User1,192.168.1.185,72ms
4,4,2023-11-20T08:40:50.697002,ERROR,ServiceB,Critical Errors,7272,User64,192.168.1.194,56ms
5,5,2023-11-20T08:40:50.705617,FATAL,ServiceA,Crashes,7983,User48,192.168.1.129,22ms
6,6,2023-11-20T08:40:50.714641,WARNING,ServiceD,Performance Warnings,3378,User40,192.168.1.56,12ms
7,7,2023-11-20T08:40:50.729253,WARNING,ServiceB,Performance Warnings,2061,User15,192.168.1.58,11ms
8,8,2023-11-20T08:40:50.742986,ERROR,ServiceD,Critical Errors,9966,User39,192.168.1.219,52ms
9,9,2023-11-20T08:40:50.759240,ERROR,ServiceB,Database Errors,3059,User57,192.168.1.97,79ms


In [5]:
# Verificando os tipos de dados
df['Message'].unique()

array(['Performance Warnings', 'File I/O', 'Critical Errors', 'Crashes',
       'Database Errors', 'Resource Warnings', 'Startup Messages',
       'Trace Information', 'Data Corruption', 'Status Updates',
       'Algorithm Steps', 'Input Validation Warnings', 'SQL Queries'],
      dtype=object)

In [6]:
df['LogLevel'].unique()

array(['WARNING', 'DEBUG', 'ERROR', 'FATAL', 'INFO'], dtype=object)

In [7]:
df['TimeTaken'].unique()

array(['28ms', '55ms', '44ms', '72ms', '56ms', '22ms', '12ms', '11ms',
       '52ms', '79ms', '64ms', '86ms', '24ms', '83ms', '61ms', '16ms',
       '87ms', '33ms', '17ms', '19ms', '94ms', '47ms', '91ms', '39ms',
       '82ms', '60ms', '65ms', '51ms', '59ms', '84ms', '93ms', '21ms',
       '43ms', '54ms', '53ms', '97ms', '14ms', '85ms', '31ms', '30ms',
       '48ms', '41ms', '27ms', '37ms', '26ms', '50ms', '32ms', '77ms',
       '96ms', '100ms', '80ms', '58ms', '74ms', '76ms', '35ms', '71ms',
       '90ms', '88ms', '92ms', '15ms', '99ms', '69ms', '62ms', '40ms',
       '67ms', '66ms', '38ms', '10ms', '49ms', '46ms', '45ms', '42ms',
       '68ms', '63ms', '18ms', '29ms', '75ms', '98ms', '81ms', '95ms',
       '25ms', '20ms', '57ms', '23ms', '13ms', '36ms', '78ms', '34ms',
       '89ms', '70ms', '73ms'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  100000 non-null  int64 
 1   Timestamp   100000 non-null  object
 2   LogLevel    100000 non-null  object
 3   Service     100000 non-null  object
 4   Message     100000 non-null  object
 5   RequestID   100000 non-null  int64 
 6   User        100000 non-null  object
 7   ClientIP    100000 non-null  object
 8   TimeTaken   100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


| **Campo**    | **Descrição** |
|--------------|---------------|
| **Timestamp** | Registra a data e hora de cada evento logado no formato `[2023-11-20T08:40:50.664842]`, fornecendo uma sequência cronológica das atividades do sistema. |
| **LogLevel**  | Indica a severidade ou importância do evento logado, classificando as entradas em níveis como `INFO`, `DEBUG`, `WARNING`, `ERROR` ou `FATAL`, oferecendo insights sobre a criticidade dos eventos. |
| **Service**   | Especifica o nome ou identificador do serviço associado a cada entrada de log, facilitando a categorização e análise dos eventos com base nos componentes modulares do sistema distribuído. |
| **Message**   | Contém informações descritivas ou detalhes relacionados ao evento logado, oferecendo contexto sobre a atividade do sistema distribuído. (ex: `Startup Messages`)|
| **RequestID** | Identifica de forma única cada requisição, permitindo rastrear e correlacionar entradas de log associadas a transações ou operações específicas. (ex: `6556`)|
| **User**      | Representa o usuário associado ao evento logado, fornecendo informações sobre a entidade que interagiu com o sistema e auxiliando na análise centrada no usuário. (ex: `User58`)|
| **ClientIP**  | Identifica de forma única o cliente ou aplicação associada ao evento, facilitando o rastreamento e análise das atividades realizadas por diferentes clientes no sistema. (ex: `192.168.1.23`)|
| **TimeTaken** | Registra a duração (ex: `17ms`), indicando o tempo necessário para completar a operação ou transação correspondente no sistema. |


### Pré-processamento

- Remover colunas que não serão utilizadas

- Converter Timestamp para datetime.

- Extrair recursos temporais: hora, minuto, dia da semana.

- Converter LogLevel em variável categórica ordenada.

- Criar variável-alvo: se nos próximos X minutos ocorre um erro grave.

In [9]:
# Novo df com apenas os logs de erro de um serviço específico
servico_mais_comum = df['Service'].value_counts().idxmax()
df = df[df['Service'] == servico_mais_comum].copy()

In [10]:
df['Service'].unique()

array(['ServiceC'], dtype=object)

In [11]:
# Drop de colunas que não serão utilizadas (adaptado para o contexto dos rappitors)
df.drop(columns=['Unnamed: 0','Service', 'User', 'RequestID', 'User', 'ClientIP', 'Message' ], inplace=True)

#### Timestamp

Extrair informações temporais que revelem padrões cíclicos (hora, dia, semana) ou frequência de eventos.

In [12]:
# Converter para datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extrair componentes temporais
df['hour'] = df['Timestamp'].dt.hour
df['day_of_week'] = df['Timestamp'].dt.dayofweek  # 0 = segunda, 6 = domingo
df['month'] = df['Timestamp'].dt.month

# Horário do dia categorizado (útil para modelos)
df['period'] = pd.cut(df['hour'], bins=[0,6,12,18,24], labels=[0,1,2,3], right=False)

# Tempo entre eventos (em segundos)
df['time_since_last'] = df['Timestamp'].diff().dt.total_seconds().fillna(0)


| Intervalo da hora | Label (`period`) |
|-------------------|------------------|
| 00h às 06h        | 0                |
| 06h às 12h        | 1                |
| 12h às 18h        | 2                |
| 18h às 24h        | 3                |

#### Categorizando LogLevel

Capturar severidade dos eventos, útil para detectar tendências de falha.

In [13]:
# Converter LogLevel em variável categórica ordenada.
loglevel_map = {
    'DEBUG': 0,
    'INFO': 1,
    'WARNING': 2,
    'ERROR': 3,
    'FATAL': 4
}

df['LogLevelEncoded'] = df['LogLevel'].map(loglevel_map)

#### TimeTaken

Transformar tempo de resposta em número para análises estatísticas.

In [14]:
# Remover 'ms' e converter para float
df['time_taken_ms'] = df['TimeTaken'].str.replace('ms', '').astype(float)

## Prever se o próximo evento será uma falha

In [15]:
# Criar variável de erro futuro (1 se o próximo log for ERROR ou FATAL, 0 caso contrário)
df['FutureError'] = df['LogLevel'].shift(-1).isin(['ERROR', 'FATAL']).astype(int)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25200 entries, 11 to 99998
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Timestamp        25200 non-null  datetime64[ns]
 1   LogLevel         25200 non-null  object        
 2   TimeTaken        25200 non-null  object        
 3   hour             25200 non-null  int32         
 4   day_of_week      25200 non-null  int32         
 5   month            25200 non-null  int32         
 6   period           25200 non-null  category      
 7   time_since_last  25200 non-null  float64       
 8   LogLevelEncoded  25200 non-null  int64         
 9   time_taken_ms    25200 non-null  float64       
 10  FutureError      25200 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(2), int32(3), int64(2), object(2)
memory usage: 1.9+ MB


In [17]:
df.head()

,Timestamp,LogLevel,TimeTaken,hour,day_of_week,month,period,time_since_last,LogLevelEncoded,time_taken_ms,FutureError
11,2023-11-20 08:40:50.798636,INFO,86ms,8,0,11,1,0.000000,1,86.0,1
14,2023-11-20 08:40:50.846785,FATAL,83ms,8,0,11,1,0.048149,4,83.0,0
15,2023-11-20 08:40:50.857131,WARNING,61ms,8,0,11,1,0.010346,2,61.0,1
16,2023-11-20 08:40:50.863332,FATAL,83ms,8,0,11,1,0.006201,4,83.0,0
19,2023-11-20 08:40:50.897574,WARNING,33ms,8,0,11,1,0.034242,2,33.0,0


In [18]:
# Importando bibliotecas para treinar o modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

### O Modelo

In [19]:
features = ['hour', 'day_of_week', 'month', 'period', 'time_since_last', 'LogLevelEncoded', 'time_taken_ms']
X = df[features]
y = df['FutureError']

In [20]:
# Divisão do dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier(n_estimators=100)

In [21]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
# Avaliação do modelo
y_pred = model.predict(X_test)

# Martiz de confusão (TP, TN, FP, FN)
print(confusion_matrix(y_test, y_pred))


[[2896  682]
 [1164  298]]


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.81      0.76      3578
           1       0.30      0.20      0.24      1462

    accuracy                           0.63      5040
   macro avg       0.51      0.51      0.50      5040
weighted avg       0.59      0.63      0.61      5040



In [24]:
import joblib

# Salva o modelo em um arquivo
joblib.dump(model, 'modelo_future_error.pkl')

['modelo_future_error.pkl']

### Exemplo real

In [25]:
# Carrega o modelo a partir do arquivo
modelo_carregado = joblib.load('modelo_future_error.pkl')

In [26]:
# Exemplo de nova linha de dados para previsão

nova_linha = pd.DataFrame([{
    "hour": 8,
    "day_of_week": 0,
    "month": 11,
    "period": 2,
    "time_since_last": 0.8,
    "LogLevelEncoded": 3,      
    "time_taken_ms": 2300.0
}])

In [27]:
# Previsão para a nova linha
previsao_nova = modelo_carregado.predict(nova_linha)
print("Previsão para nova linha:", previsao_nova)

Previsão para nova linha: [0]


In [30]:
# Exemplo de nova linha de dados para previsão

nova_linha = pd.DataFrame([{
    "hour": 14,
    "day_of_week": 2,
    "month": 4,
    "period": 1,
    "time_since_last": 0.8,
    "LogLevelEncoded": 1,      
    "time_taken_ms": 23
}])

In [31]:
# Previsão para a nova linha
previsao_nova = modelo_carregado.predict(nova_linha)
print("Previsão para nova linha:", previsao_nova)

Previsão para nova linha: [0]
